<center>
<h4>CDS 110, Lecture 8a</h4>
<font color=blue><h1>Fundamental Limits for Control of a Magnetic Levitation System</h1></font>
<h3>Richard M. Murray, Winter 2024</h3>
</center>

[Open in Google Colab](https://colab.research.google.com/drive/1MuDZfw72UkI4_Ji_AsEDTPi7IaSURsYP)

This notebook contains the code used to create the magnetic levitation example in Lecture 8-1 of CDS 110, Winter 2024.

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from math import pi
try:
  import control as ct
  print("python-control", ct.__version__)
except ImportError:
  !pip install control
  import control as ct
import control.optimal as opt
import control.flatsys as fs

The magnetic leviation system consists of a metal ball, an electromagnet, and an IR sensor:

<center><img src="https://www.cds.caltech.edu/~murray/courses/cds110/sp2024/maglev-diagram.png", alt="maglev-diagram" width=400></center>

It is governed by following equation:

$$ \ddot{z} = g - \frac{k_mk_A^2}{m}\frac{u^2}{z^2} - \frac{c}{m}\dot{z},$$

where $z$ is the vertical height of the ball and $u$ is the input current applied to the electromagnet.  The output is given by $v_{ir}$, which is the voltage measured at the IR sensor:

$$v_{ir} = k_T z + v_0 $$

In [ ]:
# System dynamics
maglev_params = {
    'kT': 613.65,      # gain between position and voltage
    'v0': -16.18,	   # voltage offset at zero position
    'm': 0.2,	       # mass of ball, kg
    'g': 9.81,         # gravitational constant
    'kA': 1,	       # electromagnet conductance
    'c': 1             # damping (added to improve visualization)
}
# gain on magnetic attractive force
maglev_params['km'] = 3.13e-3 * (maglev_params['m']/2) / maglev_params['kA']**2

def maglev_update(t, x, u, params):
    m, g, kA, km, c = map(params.get, ['m', 'g', 'kA', 'km', 'c'])
    return np.array([
        x[1],
        g - km/m * (kA * u[0])**2 / x[0]**2 - c * x[1]
    ])

def maglev_output(t, x, u, params):
    kT, v0 = map(params.get, ['kT', 'v0'])
    return np.array([kT * x[0] + v0])

maglev = ct.nlsys(
    maglev_update, maglev_output, params=maglev_params, name='maglev',
    inputs='Vu', outputs='Vy', states=['pos', 'vel']
)

In [ ]:
# Compute the equilibrium point that holds the ball at the origin
xeq, ueq = ct.find_eqpt(maglev, [0.02, 0], 0.2, y0=0)
print(f"{xeq=}, {ueq=}", end='\n----\n')

# Compute the linearization at that point
magP = ct.linearize(maglev, xeq, ueq, name='sys')
print(magP, end='\n----\n')

print("Poles:", magP.poles())
print("Zeros:", magP.zeros())

The controller for this system is implemented via an electrical circuit consisting of resistors and capacitors.  We don't show the circuit here, but just write down the model for the transfer function:

In [ ]:
# Controller (analog circuit)
k1 = 0.5				# gain set by gain pot
R1 = 22000				# Internal resistor
R2 = 22000				# Resistor plug-in
R = 2000; C = 1e-6		# RC plug-in

# Controller based on analog circuit
magC1 = -ct.tf([(R1 + R) * C, 1], [R * C, 1]) * k1 * R2/R1
magL1 = magP * magC1

We can now use a Nyquist plot to see if the controller is stabilizing:

In [ ]:
# Nyquist plot
cplt = ct.nyquist_plot([magP, magL1], label=["sys", "sys * ctrl"])

We see that the controller causes the system to have clockwise net encircelement of the origin.  Since the open loop system has one unstable pole, this gives $Z = N + P = 0$ and so the closed loop system is stable.

In [ ]:
# Bode plots
magC1.name = "ctrl"
cplt = ct.bode_plot(
    [magP, magC1, magL1], np.logspace(0, 4), initial_phase=0,
    label=['P', 'C', 'L'])
cplt.axes[0, 0].set_ylim(0.06, 1.5e1)

In [ ]:
# Sensitivity function for closed loop system/.
magS1 = ct.feedback(1, magL1, name="S1")

In [ ]:
# Step response
magT1 = ct.feedback(magL1, name="T1")
ct.step_response(magT1).plot(title="Step response for closed loop system")

In [ ]:
# Try to improve performance by increasing DC gain
# System with gain increased
magC2 = magC1*5 			                        # increased gain
magL2 = magP * magC2 			                    # loop transfer function
magS2 = ct.feedback(1, magP * magC2, name="S2") 	# sensitivity function
magT2 = ct.feedback(magP * magC2, 1, name="T2") 	# closed loop response

# System with gain increased even more
magC3 = magC1*20			                        # increased gain
magL3 = magP*magC3			                        # loop transfer function
magS3 = ct.feedback(1, magP * magC3, name="S3")	    # sensitivity function
magT3 = ct.feedback(magP * magC3, 1, name="T3")	    # closed loop response

# Plot step responses for different systems
colors = ['b', 'g', '#FF7F50']
for sys in [magT1, magT2, magT3]:
    ct.step_response(sys).plot(color=colors.pop())

# Bode plot for sensitivity function
plt.figure()
cplt = ct.bode_plot([magS1, magS2, magS3], plot_phase=False)

# Add magnitude of 1
xdata = cplt.lines[0][0][0].get_xdata()
ydata = np.ones_like(xdata)
plt.plot(xdata, ydata, color='k', linestyle='--');

In [ ]:
# Bode integral calculation
omega = np.linspace(0, 1e6, 100000)
for name, sys in zip(['C1', 'C2', 'C3'], [magS1, magS2, magS3]):
    freqresp = ct.frequency_response(sys, omega)
    bodeint = np.trapz(np.log(freqresp.magnitude), omega)
    print("Bode integral for", name, "=", bodeint)

print("pi * sum[ Re(pk) ]", pi * np.sum(magP.poles()[magP.poles().real > 0]))